In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import multiprocessing
import os
import pathlib
import xarray as xr

from tqdm.notebook import tqdm
from crims2s.util import fix_dataset_dims

In [ ]:
OBSERVATIONS = '***BASEDIR***training-output-reference/'
BENCHNMARK = '***BASEDIR***training-output-benchmark/'
CENTER = 'ecmwf'

In [ ]:
INPUT_DIR = '***BASEDIR***training-input/'
OUTPUT_DIR = '***BASEDIR***processed/training-input/'

In [ ]:
INPUT_DIR = '***BASEDIR***test-input/'
OUTPUT_DIR = '***BASEDIR***processed/test-input/'

In [ ]:
to_rechunk = ['t', 'gh', 'u', 'v']

In [ ]:
input_path = pathlib.Path(INPUT_DIR)
output_path = pathlib.Path(OUTPUT_DIR)

In [ ]:
output_path.mkdir(parents=True, exist_ok=True)

In [ ]:
input_files = []
for field in to_rechunk:
    input_files.extend([f for f in input_path.iterdir() if CENTER in f.stem and f'-{field}-' in f.stem])
    
input_files = sorted(input_files)

In [ ]:
input_files[:2]

In [ ]:
[*_, field, datestring] = input_files[0].stem.split('-')

In [ ]:
def do_one_file(f):
    new_file = output_path / f.name
    [*_, field, datestring] = f.stem.split('-')
    
    d = xr.open_dataset(f)
    
    #for level in d.plev:
    for level in [1000, 200, 850]:
        level_str = int(level)
        new_filename = f'{CENTER}-hindcast-{field}{level_str}-{datestring}.nc'
        new_path = output_path / new_filename
        
        #if new_path.is_file():
        #    print(f'Skipping {new_path} because it exists.')
        #    return
        
        subset = d.sel(plev=[level])
        #print(new_path)
        
        subset.to_netcdf(new_path, mode='w')

In [ ]:
do_one_file(input_files[0])

In [ ]:
!ls {OUTPUT_DIR}

In [ ]:
with multiprocessing.Pool(processes=6) as pool:
    for _ in tqdm(pool.imap(do_one_file, input_files), total=len(input_files)):
        pass

In [ ]:
for f in input_files:
    new_file = output_path / f.name
    datestring = f.stem.split('-')[-1]
    
    print(new_file)
    d = xr.open_dataset(f)
    d = fix_dataset_dims(d)
    
    for level in d.plev:
        level_str = int(level.item())
        new_filename = f'{CENTER}-hindcast-{FIELD}{level_str}-{datestring}.nc'
        new_path = output_path / new_filename
        subset = d.sel(plev=[level])
        print(new_path)
        
        subset.to_netcdf(new_path, mode='w')
    

In [ ]:
output_files = sorted([f for f in output_path.iterdir() if '20200102' in f.stem])

In [ ]:
xr.open_dataset(output_files[0])

In [ ]:
xr.open_mfdataset(output_files)

In [ ]:
input_files